In [1]:
import pandas as pd
import pyspark
from datetime import datetime
import os.path 
import datetime
pd.set_option('display.max_columns', 500)
import seaborn as sns
import numpy as np
import calendar
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, SVR
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.externals import joblib
from scipy.stats import norm, skew
import statsmodels.api as sm
blue = sns.color_palette('Blues')[-2]
color = sns.color_palette() 
sns.set_style('darkgrid') 
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) #Limiting floats output to 3 decimal points
import nltk
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold

/Users/yiranjing/miniconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/Users/yiranjing/miniconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


#### Load and join dataset

In [5]:
new_tech_biom = pd.read_csv("data/new_tech_biom.csv", encoding = "ISO-8859-1")
new_tech_biom = new_tech_biom.loc[:, ~new_tech_biom.columns.str.contains('^Unnamed')]
tech_nutr = pd.read_csv("data/tech_nutr.csv", encoding = "ISO-8859-1", index_col=False)
tech_nutr = tech_nutr.loc[:, ~tech_nutr.columns.str.contains('^Unnamed')]
tech_food = pd.read_csv("data/tech_food.csv", encoding = "ISO-8859-1", index_col=False )
tech_food =tech_food.loc[:, ~tech_food.columns.str.contains('^Unnamed')]

In [6]:
print("The number of rows in tech_food", len(tech_food))
print("The number of rows in new_tech_biom", len(new_tech_biom))
print("The number of rows in tech_nutr", len(tech_nutr))

The number of rows in tech_food 341897
The number of rows in new_tech_biom 12153
The number of rows in tech_nutr 12153


### Data cleaning
1. For the exercise time, if the exercise time > 1500 mins/week, trade as outlisers, set the value to 0

In [7]:
new_tech_biom["EXLWTBC"] = np.where(new_tech_biom["EXLWTBC"] >1500, 0, new_tech_biom['EXLWTBC'])
new_tech_biom["EXLWVBC"] = np.where(new_tech_biom["EXLWVBC"] >1500, 0, new_tech_biom['EXLWVBC'])
new_tech_biom["EXLWMBC"] = np.where(new_tech_biom["EXLWMBC"] >1500, 0, new_tech_biom['EXLWMBC'])

#### Join thress datasets
The join key:
1. ABSPID(unique person ID)
2. ABSHID(persons household ID)

In [8]:
# join three datasets based on ABSPID(unique person ID), ABSHID(persons household ID), 
merged_food_nutr = pd.merge(tech_nutr, tech_food, on=['ABSPID',"ABSHID"]) # inner join 
merged_food_nutr_binom = pd.merge(merged_food_nutr, new_tech_biom, on=['ABSPID'])

In [9]:
merged_food_nutr_binom.head(2)

,ABSHID,ABSPID,BMR,EIBMR1,EIBMR2,ENERGYT1,ENERGYT2,ENRGYT1,ENRGYT2,MOISTT1,MOISTT2,PROTT1,PROTT2,FATT1,FATT2,CHOWSAT1,CHOWSAT2,CHOWOAT1,CHOWOAT2,STARCHT1,STARCHT2,SUGART1,SUGART2,FIBRET1,FIBRET2,ALCT1,ALCT2,PREVAT1,PREVAT2,PROVAT1,PROVAT2,RETEQT1,RETEQT2,B1T1,B1T2,B2T1,B2T2,B3T1,B3T2,NIACINT1,NIACINT2,FOLATT1,FOLATT2,FOLICT1,FOLICT2,TOTFOLT1,TOTFOLT2,FOLEQT1,FOLEQT2,B6T1,B6T2,B12T1,B12T2,VITCT1,VITCT2,VITET1,VITET2,CALCT1,CALCT2,IODINET1,IODINET2,IRONT1,IRONT2,MAGT1,MAGT2,PHOST1,PHOST2,POTAST1,POTAST2,SELT1,SELT2,SODIUMT1,SODIUMT2,ZINCT1,ZINCT2,CAFFT1,CAFFT2,CHOLT1,CHOLT2,SATFATT1,SATFATT2,MUFATT1,MUFATT2,PUFATT1,PUFATT2,LAT1,LAT2,ALAT1,ALAT2,LCN3T1,LCN3T2,TRANST1,TRANST2,PROPER1,PROPER2,FATPER1,FATPER2,LAPER1,LAPER2,ALAPER1,ALAPER2,CHOPER1,CHOPER2,SUGPER1,SUGPER2,STARPER1,STARPER2,ALCPER1,ALCPER2,SATPER1,SATPER2,TRANPER1,TRANPER2,FIBRPER1,FIBRPER2,MONOPER1,MONOPER2,POLYPER1,POLYPER2,GRAINS1N,WHOLGR1N,REFGRA1N,VEGLEG1N,GREENS1N,VGORSV1N,STARCH1N,LEGVEG1N,OTHVEG1N,FRUIT1N,FRSHF1N,DRFR1N,FRJUIC1N,DAIRY1N,DAIRHF1N,DAIRMF1N,DAIRLF1N,MEAT1N,MEATL1N,MEATLD1N,RDMTL1N,RDMTLU1N,RDMTLP1N,RDMTN1N,RDMTNU1N,RDMTNP1N,PLTYL1N,PLTYLU1N,FISH1N,EGGS1N,LEGMT1N,NUTS1N,WATER1N,UNSAT1N,FRESUG1N,ADDSUG1N,MEATLG1N,MEATDG1N,RDMTLG1N,RDMLUG1N,RDMLPG1N,RDMTNG1N,RDMNUG1N,RDMNPG1N,PLTYLG1N,PLTLUG1N,PLTLPG1N,PLTYNG1N,PLTNUG1N,PLTNPG1N,RDMTTG1N,RDMTUG1N,PLTYTG1N,PLTYUG1N,WATERG1N,WATERG2N,GRAINS2N,VEGLEG2N,FRUIT2N,FRSHF2N,MEAT2N,DAIRY2N,MEATLD2N,MEATL2N,WATER2N,UNSAT2N,FRESUG2N,ADDSUG2N,PEFRESD1,PEADDSD1,PEFRESD2,PEADDSD2,BMR_MISS,EIBMR1_MISS,EIBMR2_MISS,GRAMWGT,FOODCODC,ENERGYWF,ENERGYNF,MOISTURE,PROTEIN,FATG,CHOWSA,CHOWOSA,STARCH,SUGARS,FIBRE,ALCOHOL,PREVITA,PROVITA,RETEQV,B1MG,B2MG,B3MG,NIACINEQ,FOLATEUG,FOLACID,TOTFOL,FOLATEQV,B6MG,B12UG,VITCMG,VITEMG,CALMG,IODINEUG,IRONMG,MAGMG,PHOSMG,POTASMG,SELUG,SODMG,ZINCMG,CAFFMG,CHOLMG,SFAG,MUFAG,PUFAG,LAG,ALAG,LCN3MG,TRANSMG,COMBCODE,EATTIMEC,EATOCC,DAYNUM,TWDIG,THRDIG,FIVDIG,DISCFLG,GRNSRV,WGSRV,...,FRSHFSRV,DRFRSRV,FRJCSRV,DAIRYSRV,HFDASRV,HFCHSSRV,HFPOWSRV,MFDASRV,MFMLKSRV,MFEVPSRV,MFCONSRV,MFCHHSRV,MFCHSSRV,MFYOGSRV,MFALTSRV,MFSNASRV,LFDASRV,LFMLKSRV,LFEVPSRV,LFCONSRV,LFCHHSRV,LFCHSSRV,LFYOGSRV,LFALTSRV,LFSNASRV,LFPOWSRV,MEATSRV,RMTLSRV,RMTLUSRV,RMTLPSRV,RMTNSRV,RMTNUSRV,RMTNPSRV,PLYLSRV,PLYLUSRV,PLYLPSRV,PLYNSRV,PLYNUSRV,PLYNPSRV,FISHSRV,EGGSSRV,LEGMTSRV,LEGBSRV,TOFUSRV,NUTSSRV,WATRSRV,UNSTSRV,UNSTSSRV,UNSTOSRV,NUTFTSRV,GRNGM,WGGM,WGBRGM,WGGNGM,WGOAGM,WGBFGM,WGSVGM,WGCRGM,WGMFGM,WGFLGM,RFGRGM,RFBRGM,RFGNGM,RFOAGM,RFBFGM,RFSVGM,RFCRGM,RFMFGM,RFFLGM,VEGGM,VGGRGM,VGORGM,STARGM,LEGVGGM,OTHVGGM,OWHLVGM,OVGJGM,FRUITGM,FRSHFGM,DRFRGM,FRJCGM,DAIRYGM,HFDAGM,HFCHSGM,HFPOWGM,MFDAGM,MFMLKGM,MFEVPGM,MFCONGM,MFCHHGM,MFCHSGM,MFYOGGM,MFALTGM,MFSNAGM,LFDAGM,LFMLKGM,LFEVPGM,LFCONGM,LFCHHGM,LFCHSGM,LFYOGGM,LFALTGM,LFSNAGM,LFPOWGM,MEATGM,RMTLGM,RMTLUGM,RMTLPGM,RMTNGM,RMTNUGM,RMTNPGM,PLYLGM,PLYLUGM,PLYLPGM,PLYNGM,PLYNUGM,PLYNPGM,FISHGM,EGGSGM,LEGMTGM,LEGBGM,TOFUGM,NUTSGM,WATRGM,UNSTGM,UNSTSGM,UNSTOGM,NUTFTGM,ADDSGGRM,FRESGGRM,MTTLSRV,MTTNSRV,RMTTSRV,RMTTUSRV,PLYTSRV,PLYTUSRV,MTTLGM,MTTNGM,RMTTGM,RMTTUGM,PLYTGM,PLYTUGM,COMBCODE_MISS,EATTIMEC_MISS,EATOCC_MISS,BMISC,AGEC,SMSBC,COBBC,FEMLSBC,PHDKGWBC,PHDCMHBC,EXLWTBC,EXLWMBC,EXLWVBC,PHDCMWBC,SF2SA1QN,INCDEC,DIABBC,HCHOLBC,HSUGBC,HYPBC,ADTOTSE,BDYMSQ04,DIASTOL,DIETQ12,DIETQ14,DIETQ5,DIETQ8,DIETRDI,SABDYMS,SEX,SLPTIME,SMKDAILY,SMKSTAT,SYSTOL,ALTNTR,ALTRESB,APOBNTR,APOBRESB,B12RESB,BIORESPC,CHOLNTR,CHOLRESB,CVDMEDST,DIAHBRSK,FASTSTAD,FOLATREB,GGTNTR,GGTRESB,GLUCFPD,GLUCFREB,HBA1PREB,HDLCHREB,LDLNTR,LDLRESB,TRIGNTR,TRIGRESB,BMISC_MISS,SMSBC_MISS,FEMLSBC_MISS,PHDKGWBC_MISS,PHDCMHBC_MISS,PHDCMWBC_MISS,INCDEC_MISS,ADTOTSE_MISS,BDYMSQ04_MISS,DIASTOL_MISS,DIETQ12_MISS,DIETQ14_MISS,DIETRDI_MISS,SABDYMS_MISS,SLPTIME_MISS,SMKDAILY_MISS,SMKSTAT_MISS,SYSTOL_MISS,ALTNTR_MISS,ALTRESB_MISS,APOBNTR_MISS,APOBRESB_MISS,B12RESB_MISS,BIORESPC_MISS,CHOLNTR_MISS,CHOLRESB_MISS,CVDMEDST_MISS,DIAHBRSK_MISS,FASTSTAD_MISS,FOLATREB_MISS,GGTNTR_MISS,GGTRESB_MISS,GLUCFPD_MISS,GLUCFREB_MISS,HBA1PREB_MISS,HDLCHREB_

In [10]:
print("The number of rows after merge food and nutr", len(merged_food_nutr))
print("The number of rows after merging three dataset", len(merged_food_nutr_binom)) # same as the size of food dataset

The number of rows after merge food and nutr 341897
The number of rows after merging three dataset 341897


In [11]:
merged_food_nutr_binom.to_csv('data/merged_food_nutr_binom.csv', index=False)